# Intro and setup

This notebook demonstrates the use of word2vec in Python using the  [gensim libraries](https://github.com/RaRe-Technologies/gensim).  Information is available on the [gensim website](https://radimrehurek.com/gensim/index.html) along with tutorials and the [API](https://radimrehurek.com/gensim/apiref.html). 

You can install them to your local machine using the command:
```
pip install --upgrade gensim
```
Of course if you are using a jupyter notebook, that doesn't mean it is installed in the correct place.  Make sure the `sys.path` command yields a location that matches the location of where the `pip install` occured.  Because the `sys.path` includes the current working directory, I use a `-t` flag to place the libraries in a location I can make the current working directory.  You could also theroretically append to `sys.path`

Word2Vec was first presented in the paper: Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781.  Extensions were presented in the paper: Mikolov, T., Sutskever, I., Chen, K., Corrado, G. S., & Dean, J. (2013). Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems (pp. 3111-3119).

In [105]:
import os
os.getcwd()

'/home/milton/programming/python/gensim'

In [106]:
# access local libraries from directory
os.chdir('/home/milton/programming/python/gensim/')

In [164]:
# gensim library also requires numpy, scipy, requests, docutils
os.listdir()

['word2vec.ipynb', '__pycache__', 'python_dateutil-2.7.3.dist-info', 'numpy', 'requests-2.19.1.dist-info', 'dateutil', 'idna', 'scipy-1.1.0.dist-info', 'jmespath-0.9.3.dist-info', 'boto', 'botocore', 'gensim-3.6.0.dist-info', 'file', 'idna-2.7.dist-info', 'jmespath', 'certifi-2018.8.24.dist-info', 'bz2file-0.98-py3.6.egg-info', 'chardet-3.0.4.dist-info', 'docutils-0.14.dist-info', 'bin', 'savedModelText8', 'urllib3-1.23.dist-info', 'smart_open', 'chardet', 'Tutorials.html', 'six-1.11.0.dist-info', 'requests', 'gensim', 'docutils', 'botocore-1.12.18.dist-info', 'boto3-1.9.18.dist-info', '.ipynb_checkpoints', 's3transfer-0.1.13.dist-info', 'bz2file.py', 'numpy-1.15.2.dist-info', '__init__.py', 'six.py', 'certifi', 'boto-2.49.0.dist-info', 'smart_open-1.7.1-py3.6.egg-info', 'scipy', 'boto3', 's3transfer', 'word2vecTalk.ipynb', 'urllib3']

# Data

In [9]:
# turn off pretty printing to get horizontal display
%pprint

Pretty printing has been turned OFF


## import data
There are lots of possible data sources.  Gensim includes specific loaders for the [Brown corpus](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.BrownCorpus) and Matt Mahoney's [text8 corpus](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus).  The Brown corpus is in the  widely used [Natural Language ToolKit](https://www.nltk.org/).  Text8 is a [cleaned text](http://mattmahoney.net/dc/textdata.html) 100MB selection of Wikipedia, getting rid of html tags, tables, spelling out numbers, etc.  It can be downloaded at http://mattmahoney.net/dc/text8.zip.

In [108]:
with open('/home/milton/data/wikidump/text8.txt', 'r') as f:
    raw_data = f.read()

In [109]:
type(raw_data)

<class 'str'>

In [110]:
# yah, it is 100 MB
len(raw_data)

100000000

In [111]:
raw_data[:1000]

' anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state the word anarchy as most anarchists use it does not imply chaos nihilism or anomie but rather a harmonious anti authoritarian society in place of what are regarded as authoritarian political structures and coercive economic institut

In [114]:
words = raw_data.split()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs


In [113]:
len(words)

17005207

In [116]:
# how many unique words?
len(set(words))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.77 µs


253854

In [117]:
list(set(words))[:100]

['perineum', 'antiproliferative', 'sconces', 'bookshelf', 'assassinate', 'macaco', 'rhost', 'targu', 'olmedo', 'virek', 'vilmos', 'tsunetoshi', 'gaggle', 'roseninsel', 'sethianism', 'ovicidal', 'intertran', 'tinet', 'merrion', 'soviettes', 'levites', 'gianfrancesco', 'melton', 'aaui', 'ametller', 'interceptor', 'telos', 'kernighan', 'hitlerjugendf', 'hannes', 'dango', 'posford', 'sail', 'bayswater', 'debswana', 'meteorology', 'gradient', 'amar', 'porsches', 'vitebsk', 'stefanitag', 'afsnee', 'zaro', 'beus', 'reformism', 'annular', 'rasputina', 'germanisches', 'daramulum', 'axopods', 'skyservice', 'ashvattama', 'yasunori', 'cheer', 'bioherbicides', 'retinoids', 'dutiable', 'llewellyn', 'entactogens', 'flatworm', 'outfit', 'multilineata', 'beignets', 'andrew', 'situs', 'hoiberg', 'chattahoochee', 'milagro', 'pittura', 'nationalversammlung', 'vliet', 'marinho', 'mcgeady', 'coulee', 'klaatu', 'sexier', 'wojskowy', 'fractint', 'jnode', 'ortm', 'seminole', 'titusville', 'reasoning', 'eximia'

# Gensim

In [2]:
import gensim

In [12]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Setup data
`gensim.models` takes a corpus broken into sentences.  I'm using the `Text8Corpus` iterator that comes as part of the `word2vec` class.  You can use any other data as long as you create an iterable to yield sentences.

In [118]:
sentences = gensim.models.word2vec.Text8Corpus('/home/milton/data/wikidump/text8.txt')

## run model

In [122]:
# run model
model = gensim.models.Word2Vec(sentences, size = 100, window = 6, min_count=5, workers=3)

2018-10-30 19:14:12,927 : INFO : collecting all words and their counts
2018-10-30 19:14:12,929 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-30 19:14:17,056 : INFO : collected 253854 word types from a corpus of 17005207 raw words and 1701 sentences
2018-10-30 19:14:17,057 : INFO : Loading a fresh vocabulary
2018-10-30 19:14:17,443 : INFO : effective_min_count=5 retains 71290 unique words (28% of original 253854, drops 182564)
2018-10-30 19:14:17,444 : INFO : effective_min_count=5 leaves 16718844 word corpus (98% of original 17005207, drops 286363)
2018-10-30 19:14:17,588 : INFO : deleting the raw counts dictionary of 253854 items
2018-10-30 19:14:17,595 : INFO : sample=0.001 downsamples 38 most-common words
2018-10-30 19:14:17,596 : INFO : downsampling leaves estimated 12506280 word corpus (74.8% of prior 16718844)
2018-10-30 19:14:17,782 : INFO : estimated required memory for 71290 words and 100 dimensions: 92677000 bytes
2018-10-30 19:14:17,783 : 

In [76]:
# save it as binary
model.save('savedModelText8')

2018-10-30 11:12:42,191 : INFO : saving Word2Vec object under savedModelText8, separately None
2018-10-30 11:12:42,198 : INFO : not storing attribute vectors_norm
2018-10-30 11:12:42,215 : INFO : not storing attribute cum_table


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.7 µs


2018-10-30 11:12:43,293 : INFO : saved savedModelText8


In [123]:
print(model)

Word2Vec(vocab=71290, size=100, alpha=0.025)


## Vocabulary

In [135]:
# get list of word vectors
words = list(model.wv.vocab)

In [138]:
# get sorted list of word vectors
words = list(model.wv.index2word)

In [139]:
len(words)

71290

In [140]:
words[:200]

['the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two', 'is', 'as', 'eight', 'for', 's', 'five', 'three', 'was', 'by', 'that', 'four', 'six', 'seven', 'with', 'on', 'are', 'it', 'from', 'or', 'his', 'an', 'be', 'this', 'which', 'at', 'he', 'also', 'not', 'have', 'were', 'has', 'but', 'other', 'their', 'its', 'first', 'they', 'some', 'had', 'all', 'more', 'most', 'can', 'been', 'such', 'many', 'who', 'new', 'used', 'there', 'after', 'when', 'into', 'american', 'time', 'these', 'only', 'see', 'may', 'than', 'world', 'i', 'b', 'would', 'd', 'no', 'however', 'between', 'about', 'over', 'years', 'states', 'people', 'war', 'during', 'united', 'known', 'if', 'called', 'use', 'th', 'system', 'often', 'state', 'so', 'history', 'will', 'up', 'while', 'where', 'city', 'being', 'english', 'then', 'any', 'both', 'under', 'out', 'made', 'well', 'her', 'e', 'number', 'government', 'them', 'm', 'later', 'since', 'him', 'part', 'name', 'c', 'century', 'through', 'because', 'x', 'university'

In [141]:
# check the index for a word
model.wv.vocab['one'].index

3

## Vectors

In [147]:
model.wv?

Type:        Word2VecKeyedVectors
String form: <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f8aa0c5f320>
File:        ~/programming/python/gensim/gensim/models/keyedvectors.py
Docstring:  
Mapping between words and vectors for the :class:`~gensim.models.Word2Vec` model.
Used to perform operations on the vectors such as vector lookup, distance, similarity etc.


In [149]:
model.wv.get_vector('one')

array([ 0.07799023, -0.31933305,  0.5495737 , -0.08271172,  0.33065677,
        0.13488497,  0.81943583,  0.11385178,  0.57544446,  0.5176106 ,
        0.46216252, -0.03041595, -1.1404728 ,  0.84579843,  2.3422627 ,
       -1.0034747 , -0.1737026 ,  0.5258449 , -0.9533849 ,  1.7051234 ,
        0.13693434,  1.1112723 , -0.5218365 , -0.47124708,  0.28735283,
       -1.9138609 ,  1.6833236 ,  1.0821652 ,  0.5419477 ,  0.8939806 ,
       -0.90089005, -1.4638139 , -3.267643  , -0.42839742,  0.3688768 ,
       -1.2353276 , -0.28666767, -0.69865596, -0.02511278, -0.5974531 ,
        0.73084515, -0.7190821 , -0.31115186, -0.6293131 ,  1.2822145 ,
        0.02650458, -0.35107258,  1.8285064 , -1.1854329 ,  2.4672053 ,
       -2.1989372 , -0.24737279,  0.32139033,  1.0759609 ,  0.13686177,
       -0.82490003, -1.7601223 , -2.66142   ,  0.66603607, -0.14208189,
        0.01184012, -1.9974691 ,  0.4787362 ,  2.374025  ,  0.7636947 ,
       -1.123626  ,  0.48831147,  0.8691541 , -1.1698463 , -0.12

In [133]:
len(model.wv.get_vector('one'))

100

### Distance from mean

In [145]:
model.wv.doesnt_match?

Signature: model.wv.doesnt_match(words)
Docstring:
Which word from the given list doesn't go with the others?

Parameters
----------
words : list of str
    List of words.

Returns
-------
str
    The word further away from the mean of all words.
File:      ~/programming/python/gensim/gensim/models/keyedvectors.py
Type:      method


In [81]:
# find word in list that is farthest from the mean
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

2018-10-30 11:22:58,007 : INFO : precomputing L2-norms of word weight vectors
/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'cereal'

In [151]:
model.wv.doesnt_match("cook janitor pilot sport teacher".split())

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'sport'

In [154]:
model.wv.doesnt_match("joy time peace angst".split())

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'time'

In [153]:
model.wv.doesnt_match("joy timely peace angst".split())

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'peace'

In [156]:
model.wv.doesnt_match("joy peace angst".split())

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'peace'

## Similarity

### Cosine similarity

In [64]:
model.wv.similarity('woman', 'man')

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.72653675

In [65]:
model.wv.similarity('woman', 'tree')

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.2849955

In [66]:
model.wv.similarity('tree', 'man')

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.22460887

In [67]:
model.wv.distance('woman', 'tree')

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.7150045037269592

In [70]:
model.wv.distance('woman', 'tree') + model.wv.similarity('woman', 'tree')

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


1.0

In [104]:
# closest by cosine similarity
model.wv.similar_by_word('woman', topn=10)

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('child', 0.803389310836792), ('girl', 0.7806227207183838), ('man', 0.7265368103981018), ('lady', 0.6917918920516968), ('mother', 0.6886497139930725), ('person', 0.6598984003067017), ('baby', 0.6564351320266724), ('herself', 0.6543370485305786), ('husband', 0.6536892652511597), ('wife', 0.6529605388641357)]

In [159]:
# closest by cosine similarity
model.wv.similar_by_word('she', topn=10)

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('he', 0.8074591159820557), ('herself', 0.7436536550521851), ('her', 0.7007310390472412), ('leto', 0.6273062825202942), ('odrade', 0.6170876026153564), ('nobody', 0.6088880896568298), ('rachel', 0.6057782769203186), ('him', 0.5911742448806763), ('baldrick', 0.5856796503067017), ('alia', 0.5799418687820435)]

In [160]:
model.wv.most_similar?

Signature: model.wv.most_similar(positive=None, negative=None, topn=10, restrict_vocab=None, indexer=None)
Docstring:
Find the top-N most similar words.
Positive words contribute positively towards the similarity, negative words negatively.

This method computes cosine similarity between a simple mean of the projection
weight vectors of the given words and the vectors for each word in the model.
The method corresponds to the `word-analogy` and `distance` scripts in the original
word2vec implementation.

Parameters
----------
positive : list of str, optional
    List of words that contribute positively.
negative : list of str, optional
    List of words that contribute negatively.
topn : int, optional
    Number of top-N similar words to return.
restrict_vocab : int, optional
    Optional integer which limits the range of vectors which
    are searched for most-similar values. For example, restrict_vocab=10000 would
    only check the first 10000 word vectors in the vocabulary order. (T

In [86]:
model.wv.most_similar(positive=['woman'], topn=10)

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('child', 0.803389310836792), ('girl', 0.7806227207183838), ('man', 0.7265368103981018), ('lady', 0.6917918920516968), ('mother', 0.6886497139930725), ('person', 0.6598984003067017), ('baby', 0.6564351320266724), ('herself', 0.6543370485305786), ('husband', 0.6536892652511597), ('wife', 0.6529605388641357)]

In [87]:
model.wv.most_similar(negative=['woman'], topn=10)

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('radiocommunications', 0.43596065044403076), ('holography', 0.4359170198440552), ('hockenheim', 0.43065929412841797), ('airfares', 0.42763426899909973), ('automation', 0.4126434624195099), ('circuits', 0.4109076261520386), ('skywalk', 0.40816304087638855), ('sanskriti', 0.4081246852874756), ('chelyabinsk', 0.4032770097255707), ('cirrus', 0.40157902240753174)]

In [88]:
model.wv.most_similar(positive=['woman', 'king'], topn=10)

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('son', 0.7318000793457031), ('queen', 0.7241658568382263), ('princess', 0.7143644690513611), ('lady', 0.7017467617988586), ('daughter', 0.6962923407554626), ('prince', 0.6942224502563477), ('wife', 0.6774018406867981), ('bride', 0.6752276420593262), ('child', 0.6582358479499817), ('husband', 0.6561128497123718)]

In [89]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=10)

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.7056832909584045), ('empress', 0.6462346315383911), ('throne', 0.6235172748565674), ('son', 0.6209807395935059), ('princess', 0.6190495491027832), ('prince', 0.6166723966598511), ('elizabeth', 0.6093157529830933), ('emperor', 0.59967041015625), ('daughter', 0.5950674414634705), ('regent', 0.5876128077507019)]

### Multiplicative combination

In [161]:
model.wv.most_similar_cosmul?

Signature: model.wv.most_similar_cosmul(positive=None, negative=None, topn=10)
Docstring:
Find the top-N most similar words, using the multiplicative combination objective,
proposed by `Omer Levy and Yoav Goldberg "Linguistic Regularities in Sparse and Explicit Word Representations"
<http://www.aclweb.org/anthology/W14-1618>`_. Positive words still contribute positively towards the similarity,
negative words negatively, but with less susceptibility to one large distance dominating the calculation.
In the common analogy-solving case, of two positive and one negative examples,
this method is equivalent to the "3CosMul" objective (equation (4)) of Levy and Goldberg.

Additional positive or negative examples contribute to the numerator or denominator,
respectively - a potentially sensible but untested extension of the method.
With a single positive example, rankings will be the same as in the default
:meth:`~gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.most_similar`.

Parameters
-

In [134]:
model.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'], topn=10)

2018-10-30 20:21:22,155 : INFO : precomputing L2-norms of word weight vectors


[('queen', 0.9123514890670776), ('empress', 0.8933207988739014), ('princess', 0.8767482042312622), ('daughter', 0.873872697353363), ('elizabeth', 0.8636165857315063), ('son', 0.8565146923065186), ('throne', 0.8526657819747925), ('wife', 0.8484319448471069), ('matilda', 0.8467848896980286), ('mary', 0.8464444875717163)]

In [89]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=10)

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.7056832909584045), ('empress', 0.6462346315383911), ('throne', 0.6235172748565674), ('son', 0.6209807395935059), ('princess', 0.6190495491027832), ('prince', 0.6166723966598511), ('elizabeth', 0.6093157529830933), ('emperor', 0.59967041015625), ('daughter', 0.5950674414634705), ('regent', 0.5876128077507019)]

In [162]:
model.wv.most_similar_cosmul(positive=['woman', 'king'], topn=10)

[('princess', 0.5984137654304504), ('queen', 0.5973829030990601), ('son', 0.5964484214782715), ('lady', 0.5870946049690247), ('lover', 0.5867147445678711), ('daughter', 0.5812955498695374), ('wife', 0.572877824306488), ('bride', 0.5694237351417542), ('consort', 0.5694111585617065), ('heir', 0.5631678700447083)]

In [163]:
model.wv.most_similar(positive=['woman', 'king'], topn=10)

/home/milton/programming/python/gensim/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.7054295539855957), ('princess', 0.6987999677658081), ('son', 0.6968942880630493), ('lover', 0.6871877908706665), ('lady', 0.6871306896209717), ('daughter', 0.6713993549346924), ('wife', 0.6602763533592224), ('bride', 0.6555007696151733), ('man', 0.6524356007575989), ('consort', 0.6515148282051086)]


[<img style="float: left;" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png">](http://creativecommons.org/licenses/by-sa/4.0/)  

Licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/).